# Prediciendo ventas de productos

En este ejercicio continuaremos viendo el dataset visto en la notebook anterior. Queremos predecir las ventas del producto y nuestro conjunto de datos tiene las ventas en 200 mercados, y el presupuesto dedicado en publicidad en 3 medios: TV, radio y diario.

Carguemos este dataset y algunas librerías:

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns; sns.set()

In [4]:
df = pd.read_csv('data/advertising_esp.csv')

In [5]:
df

,tv,radio,periodico,ventas
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


Como habiamos visto antes la relación parece ser lineal con la variable TV, es decir cuanta plata se invirtió ahí. Crear un `X` dataframe que tenga solo las variables TV, Radio y Newspaper, y crear una variable `y` que tenga solo el dato de las ventas.

In [6]:
# Completar
X = df[['tv', 'radio', 'periodico']]
y = df['ventas']

## Ejercicio 1 - Train Test Split

a) En este ejericio elaborar una separación de datos en entrenamiento, dejando un 25% de datos para prueba. Para eso utilicen `train_test_split` de Scikit-Learn basandose en el ejemplo presente en la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), deberían crear una variable `X_train`, `X_test`, con los features, y con los targets `y_train`, e `y_test`. ¿Qué tamaño tiene cada conjunto? ¿El conjunto que crearon tiene el mismo orden de filas?


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.75,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

In [35]:
print (f'tamaño de X_train {X_train.shape}')
print (f'tamaño de X_test {X_test.shape}')
print (f'tamaño de y_train {y_train.shape}')
print (f'tamaño de y_test {y_test.shape}')

tamaño de X_train (150, 3)
tamaño de X_test (50, 3)
tamaño de y_train (150, 1)
tamaño de y_test (50, 1)


b) Quedarse solo con la columna que corresponde a TV, y ajustar un modelo de regresion lineal al conjunto de entrenamiento con intercept. Tambien construir una matriz que tenga solo la columna de TV en el conjunto de test. 

In [8]:
from sklearn.linear_model import LinearRegression

# Se elimina la columna periodico del conjunto de train y test
X_train_tv = X_train.drop(columns = ['radio','periodico'])  #dos formas de hacerlo, o eliminamos 
X_test_tv  = X_test[['tv']]                                 #o nos quedamos con la/las columnas que queremos
#X_test_tv  = X_test[:,['tv']]

In [9]:
print (type(X_train_tv))
print (type(X_test_tv))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [10]:
# Crear y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train_tv, y_train)

# Ver coeficientes del modelo
print(f"Intercepto: {modelo.intercept_}")
print(f"Coeficiente de TV: {modelo.coef_[0]}")


Intercepto: [7.07799047]
Coeficiente de TV: [0.04844509]


c) Ahora vamos a querer evaluar nuestro modelo en nuestro conjunto de testeo, para eso tenemos que calcular las predicciones del modelo $\hat{y}$, y para esto usamos el método `predict` sobre todo el conjunto de testeo.

`y_predicted = modelo.predict(X_test)`

Luego, usando los valores reales de `y` en el conjunto de test, nos interesa calcular el error de nuestro modelo en datos que no vio, justamente el conjunto de test. Para eso vamos a calcular uno de los errores más populares, el [Error Cuadrático Medio](https://es.wikipedia.org/wiki/Error_cuadr%C3%A1tico_medio):

$ ECM = \dfrac{1}{n} \sum (y_i - \hat{y}_i)^2$

Es decir que el ECM es el promedio de la diferencia entre los vectores `y_test` e `y_predict` elevadas al cuadrado. Calcular el ECM usando solo herramientas de `numpy`:

In [11]:
# calculamos las predicciones sobre test
y_predict = modelo.predict(X_test_tv)

# calculamos el cuadrado de las diferencias de estos vectores
ecm = np.mean ((y_test - y_predict) **2 )

# calculamos el promedio usando mean
print('El Error Cuadrático Medio (ECM/ RMSE) fue de: ', ecm)

El Error Cuadrático Medio (ECM/ RMSE) fue de:  11.235412479327138


d) Podemos usar el ECM que nos brinda SkLearn: [RMS](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html?highlight=rms). Calcularlo el ECM para cada conjunto, entrenamiento y test. ¿En que conjunto rindió mejor nuestro modelo?

In [39]:
from sklearn.metrics import root_mean_squared_error

# Predicciones en el conjunto de entrenamiento
y_train_pred = modelo.predict(X_train_tv)

# Calcular ECM en entrenamiento
ecm_train = root_mean_squared_error(y_train, y_train_pred)
print(f"ECM en entrenamiento: {ecm_train:.4f}")

ECM en entrenamiento: 3.2129


In [40]:
# Predicciones en el conjunto de test
y_test_pred = modelo.predict(X_test_tv)

# Calcular ECM en test
ecm_test = root_mean_squared_error(y_test, y_test_pred)
print(f"ECM en test: {ecm_test:.4f}")

ECM en test: 3.3519


## Ejercicio 2 - Probando otras variables

Ajustar un modelo lineal para cada variable sobre el conjunto que entrenamiento y calcular el error sobre cada conjunto para cada modelo. Comparar el rendimiento de cada modelo en ambos conjuntos, ¿cuál fue el mejor modelo?. Por lo visto en clase, ¿era esperable este resultado?

In [ ]:
from sklearn.metrics import mean_squared_error

# Completar

# sugerencia (pueden no usarla)
def regresion_ls(X_train:pd.DataFrame, 
                 X_test :pd.DataFrame,
                 y_train:pd.Series, 
                 y_test :pd.Series,
                 var: str):
  """
  Input
    X, y: Datos
    var: Nombre del feature a utilizar para ajustar un modelo de regresion simple
  Output
    El modelo y métrica
  """
  ...
  
  return modelo

...

¿Cuál es el peso asignado a cada variable por nuestros modelos? ¿Utilizaría solo este dato para hacer una recomendación sobre donde invertir de los tres medios?

In [ ]:
# Completar
...

# Haciendo regresiones en conjuntos de juguete

Scikit-Learn es una librería muy versátil que nos brinda herramientas no solo relacionados a los tipos de modelos, sino que tambíen nos deja crear datasets de juguete según la tarea que nos interese, en este caso la regresión. Para eso contamos con la función [`make_regresion`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html#sklearn.datasets.make_regression), donde no solo obtenemos las features y los targets, sino que también nos devuelve el coeficiente utilizado para generar este conjunto. 

In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# Ejemplo
n_samples_train, n_samples_test, n_features = 4000, 1000, 1

X, y, coef = make_regression(
    n_samples=n_samples_train + n_samples_test,
    n_features=n_features,
    n_informative=1,
    shuffle=False,
    noise=10.0,
    coef=True,
    bias = -7,
    random_state = 2022
)

plt.scatter(X,y);
print(coef)

Esto nos crea un dataset donde con la cantidad de muestras solicitadas, la cantidad de features, pudiendo controlar el ruido que se utiliza para generar estas instancias. Vamos a utilizar este dataset para evaluar nuestros modelos de regresión, aprovechando que sabemos el *coeficiente* que se utilizó para generar los datos.

## Ejercicio 1 - Toy Dataset

A partir del X e y creados anteriormente, seleccione los últimos 1000 datos para el conjunto de testeo y los primeros 4000 para entrenamiento. Ajuste un modelo lineal con intercept y reporte el *error cuadrático* tanto en entrenamiento como en test. Reporten el error cuadrático en test.

In [ ]:
# Completar
X_train = ...
y_train = ...
X_test = ...
y_test = ...

...

¿Qué tan parecidos quedaron los coeficientes de nuestro modelo en comparación de los que se usaron para generar nuestro dataset?

In [ ]:
# Completar
...

Graficar los datos junto a nuestro modelo. 

In [ ]:
# Completar

# para graficar la recta
xfit = np.linspace(-4, 4, 1000)
yfit = ...

plt.scatter(X_train, y_train, label = 'train')
plt.scatter(X_test, y_test, label = 'test')
plt.plot(xfit, yfit, 'r', label = 'modelo');
plt.legend();

## Ejercicio 2 - Comparando nuestros modelos con $R^2$

$R^2$ es una métrica que nos permite evaluar cuanta variabilidad del conjunto original de datos camptura nuestro modelo. Se lo conoce también como el [*coeficiente de determinación*](https://es.wikipedia.org/wiki/Coeficiente_de_determinaci%C3%B3n), y sklearn tiene su [implementación](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html?highlight=r2#sklearn.metrics.r2_score). A diferencia del ECM, no se trata de una función de error, que mide que tan lejos esta nuestro modelo de los datos reales (errores más chicos es mejor), sino de un *score* o puntaje, es decir que más grande es mejor y lo máximo que pude valer es 1.

$ R^2 = 1 - \frac{\text{variación no explicada}}{\text{variación total}} = 1 - \dfrac{(\sum y_i - \hat{y}_i)^2}{(\sum y_i - \bar{y})^2}$


Calcular el valor de $R^2$ para nuestro modelo:

In [ ]:
from sklearn.metrics import r2_score

# Completar
r2 = ...
print(r2)

## Ejercicio 3 - Efecto del conjunto de entrenamiento: Learning Curves

Muchas veces escucharemos aseverar que tener más datos es mejor, pero eso no es necesariamente cierto. Para ver si nuestro modelo necesita mas datos para mejorar su rendimiento podemos armar lo que se conoce como *curvas de aprendizaje* o *Learning curves*.

Básicamente lo que haremos es empezar de un conjunto de entrenamiento muy pequeño e iremos incrementando la cantidad de datos del mismo, para ver si el rendimiento de cada modelo mejor o no al incorporar más datos. Scikit Learn nos provee una [función](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html#sklearn.model_selection.learning_curve) para hacerlo, sin embargo vamos a hacer algo similar pero de manera manual.

Completar el código siguiente para calcular un modelo para cada subconjunto de entrenamiento considerado, en el cual nos iremos quedando con distintos porcentajes de entrenamiento desde el 5% al 100%. Guardarse los $R^2$ en entrenamiento y testeo para cada modelo y finalmente realizar un gráfico donde veamos el error en función del porcentaje total de datos usado.

In [ ]:
# Completar - 

train_errs = []
test_errs  = []

...

plt.plot(np.linspace(0.05,1,20), train_errs)
plt.plot(np.linspace(0.05,1,20), test_errs)
plt.legend(['train', 'test']);

Vemos finalmente que nuestro modelo mejora con más datos de entrenamiento su rendimiento en entrenamiento, como era esperable. Sin embargo, satura rapidamente el rendimiento en test.

*Aclaración*: Lo que acabamos de hacer no es exactamente una curva de aprendizaje, ya que deberiamos, para cada percentil, poder ajustar muchos conjuntos aleatorio con esa cantida de casos. Esto al menos nos garantizará poder estimar los desvíos para poder tener mayor rigurosidad estadística.

Para más detalles analizar este [ejemplo](https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html).